In [114]:
import datetime as dt
from datetime import date
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn import preprocessing

# Data Input, EDA & Preprocessing

## Individual Data Input

In [115]:
price_vol = pd.read_csv('../data/Price & Volume BTC.csv')
tweets = pd.read_csv('../data/Tweets.csv')
funding_rates_1600 = pd.read_csv('../data/Funding Rates BTCUSDT 1600.csv')
funding_rates_0800 = pd.read_csv('../data/Funding Rates BTCUSDT 0800.csv')
funding_rates_0000 = pd.read_csv('../data/Funding Rates BTCUSDT 0000.csv')
google = pd.read_csv('../data/Google.csv')
transactions = pd.read_csv('../data/Transactions.csv')
unique_addresses = pd.read_csv('../data/Unique Addresses.csv')
active_addresses = pd.read_csv('../data/Active Addresses.csv')

## Data Compilation

In [116]:
# get daily ave value for twitter attributes
tweets = tweets.groupby(by="Time").mean()
tweets = tweets.reset_index()

df = pd.merge(price_vol, funding_rates_0000, on=['Time'])
df = pd.merge(df, funding_rates_0800, on=['Time'])
df = pd.merge(df, funding_rates_1600, on=['Time'])
df = pd.merge(df, google, on=['Time'])
df = pd.merge(df, active_addresses, on=['Time'])
df = pd.merge(df, unique_addresses, on=['Time'])
df = pd.merge(df, transactions, on=['Time'])
df = pd.merge(tweets, df, how="outer")
df.head()

,Time,Favorites,Retweets,Compound,Negative,Positive,Neutral,Open,Close,High,Low,Volume,Funding Rate 0000,Funding Rate 0800,Funding Rate 1600,Bitcoin_SVI,Cryptocurrency_SVI,No. of Active Addresses,No. of Unique Addresses,Transactions
0,2020-03-12,5.0,2.0,0.0772,0.091,0.100,0.809,7933.2,4802.9,7960.4,4448.6,55248355.79,0.02%,0.01%,0.01%,22.83,6.20,909601,653156.0,349978.0
1,2020-03-17,98.0,96.0,0.4019,0.000,0.114,0.886,5028.8,5316.7,5532.0,4926.1,18269210.08,-0.09%,-0.02%,0.00%,10.21,3.39,765477,540902.0,278326.0
2,2020-03-20,125.0,95.0,0.4767,0.000,0.146,0.854,6162.2,6204.4,6901.7,5661.9,37752569.82,-0.03%,-0.02%,0.00%,10.04,3.56,734614,564748.0,284594.0
3,2020-03-30,3.0,0.0,0.0803,0.061,0.071,0.869,5881.3,6394.9,6600.0,5860.0,17904828.40,-0.03%,-0.06%,-0.03%,5.92,3.63,770915,535787.0,276828.0
4,2020-03-31,186.0,142.0,0.6597,0.000,0.278,0.722,6395.0,6409.7,6520.0,6288.0,13159710.94,-0.01%,-0.01%,-0.02%,5.45,3.54,816889,546702.0,290227.0


In [117]:
print("Type of variables: ", "\n", df.dtypes)

Type of variables:  
 Time                        object
Favorites                  float64
Retweets                   float64
Compound                   float64
Negative                   float64
Positive                   float64
Neutral                    float64
Open                       float64
Close                      float64
High                       float64
Low                        float64
Volume                     float64
Funding Rate 0000           object
Funding Rate 0800           object
Funding Rate 1600           object
Bitcoin_SVI                float64
Cryptocurrency_SVI         float64
No. of Active Addresses      int64
No. of Unique Addresses    float64
Transactions               float64
dtype: object


# EDA & Preprocessing

In [118]:
# Select features, the target output is'Close'
feature_pool = ['Favorites','Retweets','Compound','Negative','Positive',\
                'Neutral','Open', 'High','Low',\
                'Volume','Funding Rate 0000','Funding Rate 0800','Funding Rate 1600','Bitcoin_SVI',\
                'Cryptocurrency_SVI','No. of Active Addresses','No. of Unique Addresses','Transactions']

output = 'Close'

## Data Type Transformation

In [119]:
# Transform object type of "Funding Rate 0000", "Funding Rate 0800" and " Funding Rate 1600" to numerical type
df['Funding Rate 0000'] = df['Funding Rate 0000'].map(lambda x: float('nan') if pd.isnull(x) else float(x.replace('%','')))
df['Funding Rate 0800'] = df['Funding Rate 0800'].map(lambda x: float('nan') if pd.isnull(x) else float(x.replace('%','')))
df['Funding Rate 1600'] = df['Funding Rate 1600'].map(lambda x: float('nan') if pd.isnull(x) else float(x.replace('%','')))

## Train-Test Split

In [120]:
X, y = df.drop(output, axis=1), df[output]

# Split the data: train set: test set using = 70/30. 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)
print(y_train.sum()/y_train.count(),y_test.sum()/y_test.count())

31261.829549902155 33665.843378995436


## Handle Missing Data

In [121]:
# Count missing value
print(df.isna().sum())

# check the consistency of columns with missing value between train set and the whole set
print(df[feature_pool].isnull().any().tolist()==X_train.isnull().any().tolist())
print(df[output].isnull().any().tolist()==y_train.isnull().any().tolist())

Time                         0
Favorites                  458
Retweets                   458
Compound                   458
Negative                   458
Positive                   458
Neutral                    458
Open                         0
Close                        0
High                         0
Low                          0
Volume                       0
Funding Rate 0000            0
Funding Rate 0800            0
Funding Rate 1600            0
Bitcoin_SVI                  0
Cryptocurrency_SVI           0
No. of Active Addresses      0
No. of Unique Addresses      2
Transactions                 0
dtype: int64
False
True


### Monthly Sentiment Analysis

In [122]:
tweets.insert(1, "Month", "NaN")

tweets['Month'] = tweets['Time'].str[:7]
tweets.head(10)

,Time,Month,Favorites,Retweets,Compound,Negative,Positive,Neutral
0,2020-03-12,2020-03,5.0,2.0,0.0772,0.091,0.100,0.809
1,2020-03-17,2020-03,98.0,96.0,0.4019,0.000,0.114,0.886
2,2020-03-20,2020-03,125.0,95.0,0.4767,0.000,0.146,0.854
3,2020-03-30,2020-03,3.0,0.0,0.0803,0.061,0.071,0.869
4,2020-03-31,2020-03,186.0,142.0,0.6597,0.000,0.278,0.722
5,2020-04-01,2020-04,171.0,120.0,0.5267,0.061,0.184,0.754
6,2020-04-16,2020-04,419.0,94.0,0.0000,0.000,0.000,1.000
7,2020-04-22,2020-04,73.0,18.0,0.3182,0.000,0.084,0.916
8,2020-04-30,2020-04,994.0,153.0,0.0000,0.000,0.000,1.000
9,2020-05-09,2020-05,35.0,2.0,0.0000,0.000,0.000,1.000


In [123]:
tweets = tweets.groupby(by="Month").mean()
tweets = tweets.reset_index()

In [124]:
X_train.insert(1, "Month", "NaN")
X_train['Month'] = X_train['Time'].str[:7]

X_test.insert(1, "Month", "NaN")
X_test['Month'] = X_test['Time'].str[:7]

In [125]:
df_X_train = pd.DataFrame(columns = ['Time', 'Month', 'Favorites', 'Retweets', 'Compound', \
                                 'Negative', 'Positive', 'Neutral', 'Open', 'High', \
                                 'Low', 'Volume', 'Funding Rate 0000', 'Funding Rate 0800', \
                                 'Funding Rate 1600', 'Bitcoin_SVI', 'Cryptocurrency_SVI',  'No. of Active Addresses', \
                                 'No. of Unique Addresses', 'Transactions'])

for i, row in X_train.iterrows():
    if (pd.isna(X_train.loc[i].at["Favorites"])):
        data = tweets[tweets['Month'] == X_train.loc[i].at["Month"]]
        new_row = {'Time': X_train.loc[i].at["Time"], 'Month': X_train.loc[i].at["Month"], 'Favorites': data.Favorites.values[0], 'Retweets': data.Retweets.values[0], 'Compound': data.Compound.values[0], \
                   'Negative': data.Negative.values[0], 'Positive': data.Positive.values[0], 'Neutral': data.Neutral.values[0], 'Open': X_train.loc[i].at["Open"], \
                   'High': X_train.loc[i].at["High"], 'Low': X_train.loc[i].at["Low"], 'Volume': X_train.loc[i].at["Volume"], 'Funding Rate 0000': X_train.loc[i].at["Funding Rate 0000"], \
                   'Funding Rate 0800': X_train.loc[i].at["Funding Rate 0800"], 'Funding Rate 1600': X_train.loc[i].at["Funding Rate 1600"], 'Bitcoin_SVI': X_train.loc[i].at["Bitcoin_SVI"], \
                   'Cryptocurrency_SVI': X_train.loc[i].at["Cryptocurrency_SVI"], 'No. of Active Addresses': X_train.loc[i].at["No. of Active Addresses"], 'No. of Unique Addresses': X_train.loc[i].at["No. of Unique Addresses"], \
                   'Transactions': X_train.loc[i].at["Transactions"]}
        df_X_train = df_X_train.append(new_row, ignore_index=True)
    else:
        new_row = {'Time': X_train.loc[i].at["Time"], 'Month': X_train.loc[i].at["Month"], 'Favorites': X_train.loc[i].at["Favorites"], 'Retweets': X_train.loc[i].at["Retweets"], 'Compound': X_train.loc[i].at["Compound"], \
                   'Negative': X_train.loc[i].at["Negative"], 'Positive': X_train.loc[i].at["Positive"], 'Neutral': X_train.loc[i].at["Neutral"], 'Open': X_train.loc[i].at["Open"],\
                   'High': X_train.loc[i].at["High"], 'Low': X_train.loc[i].at["Low"], 'Volume': X_train.loc[i].at["Volume"], 'Funding Rate 0000': X_train.loc[i].at["Funding Rate 0000"], \
                   'Funding Rate 0800': X_train.loc[i].at["Funding Rate 0800"], 'Funding Rate 1600': X_train.loc[i].at["Funding Rate 1600"], 'Bitcoin_SVI': X_train.loc[i].at["Bitcoin_SVI"], \
                   'Cryptocurrency_SVI': X_train.loc[i].at["Cryptocurrency_SVI"], 'No. of Active Addresses': X_train.loc[i].at["No. of Active Addresses"], 'No. of Unique Addresses': X_train.loc[i].at["No. of Unique Addresses"], \
                   'Transactions': X_train.loc[i].at["Transactions"]}
        df_X_train = df_X_train.append(new_row, ignore_index=True)


In [126]:
df_X_test = pd.DataFrame(columns = ['Time', 'Month', 'Favorites', 'Retweets', 'Compound', \
                                 'Negative', 'Positive', 'Neutral', 'Open', 'High', \
                                 'Low', 'Volume', 'Funding Rate 0000', 'Funding Rate 0800', \
                                 'Funding Rate 1600', 'Bitcoin_SVI', 'Cryptocurrency_SVI',  'No. of Active Addresses', \
                                 'No. of Unique Addresses', 'Transactions'])

for i, row in X_test.iterrows():
    if (pd.isna(X_test.loc[i].at["Favorites"])):
        data = tweets[tweets['Month'] == X_test.loc[i].at["Month"]]
        new_row = {'Time': X_test.loc[i].at["Time"], 'Month': X_test.loc[i].at["Month"], 'Favorites': data.Favorites.values[0], 'Retweets': data.Retweets.values[0], 'Compound': data.Compound.values[0], \
                   'Negative': data.Negative.values[0], 'Positive': data.Positive.values[0], 'Neutral': data.Neutral.values[0], 'Open': X_test.loc[i].at["Open"], \
                   'High': X_test.loc[i].at["High"], 'Low': X_test.loc[i].at["Low"], 'Volume': X_test.loc[i].at["Volume"], 'Funding Rate 0000': X_test.loc[i].at["Funding Rate 0000"], \
                   'Funding Rate 0800': X_test.loc[i].at["Funding Rate 0800"], 'Funding Rate 1600': X_test.loc[i].at["Funding Rate 1600"], 'Bitcoin_SVI': X_test.loc[i].at["Bitcoin_SVI"], \
                   'Cryptocurrency_SVI': X_test.loc[i].at["Cryptocurrency_SVI"], 'No. of Active Addresses': X_test.loc[i].at["No. of Active Addresses"], 'No. of Unique Addresses': X_test.loc[i].at["No. of Unique Addresses"], \
                   'Transactions': X_test.loc[i].at["Transactions"]}
        df_X_test = df_X_test.append(new_row, ignore_index=True)
    else:
        new_row = {'Time': X_test.loc[i].at["Time"], 'Month': X_test.loc[i].at["Month"], 'Favorites': X_test.loc[i].at["Favorites"], 'Retweets': X_test.loc[i].at["Retweets"], 'Compound': X_test.loc[i].at["Compound"], \
                   'Negative': X_test.loc[i].at["Negative"], 'Positive': X_test.loc[i].at["Positive"], 'Neutral': X_test.loc[i].at["Neutral"], 'Open': X_test.loc[i].at["Open"], \
                   'High': X_test.loc[i].at["High"], 'Low': X_test.loc[i].at["Low"], 'Volume': X_test.loc[i].at["Volume"], 'Funding Rate 0000': X_test.loc[i].at["Funding Rate 0000"], \
                   'Funding Rate 0800': X_test.loc[i].at["Funding Rate 0800"], 'Funding Rate 1600': X_test.loc[i].at["Funding Rate 1600"], 'Bitcoin_SVI': X_test.loc[i].at["Bitcoin_SVI"], \
                   'Cryptocurrency_SVI': X_test.loc[i].at["Cryptocurrency_SVI"], 'No. of Active Addresses': X_test.loc[i].at["No. of Active Addresses"], 'No. of Unique Addresses': X_test.loc[i].at["No. of Unique Addresses"], \
                   'Transactions': X_test.loc[i].at["Transactions"]}
        df_X_test = df_X_test.append(new_row, ignore_index=True)


In [127]:
X_test = df_X_test
X_test = X_test.drop(['Time'], axis=1)
X_test = X_test.drop(['Month'], axis=1)
X_test

,Favorites,Retweets,Compound,Negative,Positive,Neutral,Open,High,Low,Volume,Funding Rate 0000,Funding Rate 0800,Funding Rate 1600,Bitcoin_SVI,Cryptocurrency_SVI,No. of Active Addresses,No. of Unique Addresses,Transactions
0,1626.600000,264.200000,-0.055560,0.069000,0.026800,0.904200,46678.5,47525.7,45562.7,4.528671e+08,0.01,0.01,0.01,13.34,29.15,1015255,678671.0,266947.0
1,1038.000000,164.000000,0.969600,0.000000,0.372000,0.628000,48815.3,50380.8,48595.0,2.520587e+08,0.03,0.04,0.05,11.20,26.81,1026339,714181.0,289900.0
2,303.750000,79.000000,0.227450,0.000000,0.065250,0.934750,9206.6,9218.7,9132.3,1.624227e+07,0.01,0.01,0.01,4.29,4.96,940265,644972.0,309747.0
3,3361.500000,543.500000,0.285650,0.023500,0.086500,0.890000,36669.3,37549.0,35703.8,3.790854e+08,0.01,0.01,0.01,17.75,27.60,892344,653685.0,265128.0
4,1769.357143,184.857143,0.241271,0.002214,0.134714,0.863143,38160.7,38255.9,30425.1,5.255061e+08,0.05,0.09,0.08,44.87,14.71,1211948,801285.0,333958.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,1870.122024,278.058036,0.376858,0.009379,0.122107,0.868536,45170.5,46443.0,44738.1,1.884884e+08,0.01,0.01,0.01,9.72,21.68,705573,495873.0,192691.0
215,2672.116667,532.600000,0.161596,0.042489,0.096383,0.861117,54948.5,56505.5,54095.0,4.156828e+08,0.02,0.01,0.02,13.32,25.40,803826,536877.0,211506.0
216,1893.148148,241.161111,0.417171,0.030372,0.158927,0.810756,56678.4,57990.0,48352.8,5.226733e+08,0.03,0.02,0.03,22.03,57.53,1245595,852846.0,300452.0
217,189.666667,35.333333,0.364667,0.041000,0.126333,0.832667,44374.8,45831.9,43165.3,5.270115e+08,0.01,0.01,0.00,14.07,29.12,1029721,673697.0,260813.0


In [128]:
X_train = df_X_train
X_train = X_train.drop(['Time'], axis=1)
X_train = X_train.drop(['Month'], axis=1)
X_train

,Favorites,Retweets,Compound,Negative,Positive,Neutral,Open,High,Low,Volume,Funding Rate 0000,Funding Rate 0800,Funding Rate 1600,Bitcoin_SVI,Cryptocurrency_SVI,No. of Active Addresses,No. of Unique Addresses,Transactions
0,505.000000,44.900000,0.409820,0.013700,0.131100,0.855200,49848.4,50660.8,46556.7,3.793275e+08,0.02,0.01,0.01,25.96,51.34,953308,646899.0,232099.0
1,1199.634921,155.000000,0.385409,0.007032,0.134857,0.858127,56152.7,57583.4,54205.2,4.326343e+08,0.01,0.01,0.01,19.57,40.81,884672,674744.0,208413.0
2,49.000000,30.000000,0.345050,0.000000,0.115500,0.884500,56245.0,57858.9,55320.0,4.671002e+08,0.01,0.01,0.01,16.52,48.95,909010,700954.0,283401.0
3,1769.357143,184.857143,0.241271,0.002214,0.134714,0.863143,36750.5,40350.0,35700.0,2.453187e+08,0.15,0.10,0.14,33.19,21.43,1292297,940647.0,401744.0
4,3114.875000,508.375000,0.234825,0.021500,0.159750,0.818750,37166.8,38006.0,36162.0,4.369649e+08,-0.01,0.00,0.01,13.81,25.39,953545,680599.0,264609.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506,128.000000,30.000000,0.106867,0.022667,0.055000,0.922333,39117.3,39864.0,36977.4,3.883689e+08,0.01,0.01,0.00,12.96,21.07,768705,583361.0,211164.0
507,1751.500000,283.750000,0.350175,0.000000,0.102500,0.897500,48873.4,49435.1,47512.0,3.903172e+08,0.01,0.01,0.01,12.81,29.12,873836,664445.0,256206.0
508,254.666667,37.833333,0.117625,0.049250,0.123000,0.827750,19160.0,19480.0,18479.7,4.867373e+07,0.08,0.08,0.04,14.65,9.20,1127776,812699.0,361990.0
509,487.428571,56.571429,0.133693,0.058571,0.110143,0.831286,23813.1,24125.0,22615.8,7.142792e+07,0.02,0.01,0.01,10.88,7.71,1138302,782736.0,338463.0


### Unique Addresses

In [129]:
KNN_miss_filling = KNNImputer(n_neighbors=2).fit(X_train)
X_train = pd.DataFrame(KNN_miss_filling.transform(X_train), columns=feature_pool)
X_test = pd.DataFrame(KNN_miss_filling.transform(X_test), columns=feature_pool)

In [130]:
print(X_train.isna().sum())
print(X_test.isna().sum())

Favorites                  0
Retweets                   0
Compound                   0
Negative                   0
Positive                   0
Neutral                    0
Open                       0
High                       0
Low                        0
Volume                     0
Funding Rate 0000          0
Funding Rate 0800          0
Funding Rate 1600          0
Bitcoin_SVI                0
Cryptocurrency_SVI         0
No. of Active Addresses    0
No. of Unique Addresses    0
Transactions               0
dtype: int64
Favorites                  0
Retweets                   0
Compound                   0
Negative                   0
Positive                   0
Neutral                    0
Open                       0
High                       0
Low                        0
Volume                     0
Funding Rate 0000          0
Funding Rate 0800          0
Funding Rate 1600          0
Bitcoin_SVI                0
Cryptocurrency_SVI         0
No. of Active Addresses    0
N

In [131]:
X_test.to_csv('../data/X_test.csv', index=False)
X_train.to_csv('../data/X_train.csv', index=False)
y_train.to_csv('../data/Y_train.csv', index=False)
y_test.to_csv('../data/Y_test.csv', index=False)

## Preprocessing

In [132]:
# Standardized features:
standardized_features = ['Favorites','Retweets','Compound','Negative','Positive','Neutral','Bitcoin_SVI']     
non_standardized_features = list(set(feature_pool)-set(standardized_features))
    
# Get the scaler based on train set
scaler = preprocessing.StandardScaler().fit(X_train[standardized_features])
print('The mean and standard deviation of training set:', scaler.mean_, scaler.scale_)

# Standardize the features of train and test sets by the scaler of training set
X_train_std=pd.DataFrame(scaler.transform(X_train[standardized_features]))  # transform() return 'numpy.ndarray', not 'DataFrame' or 'Series'
X_train_nstd=pd.DataFrame(X_train[non_standardized_features])

X_test_std=pd.DataFrame(scaler.transform(X_test[standardized_features]))  # return 'numpy.ndarray', not 'DataFrame' or 'Series'
X_test_nstd=pd.DataFrame(X_test[non_standardized_features])

# Rename the columns of X_train_std and X_test_std
X_train_std.columns = X_train_std.columns.map(lambda x: standardized_features[x])
X_test_std.columns = X_test_std.columns.map(lambda x: standardized_features[x])

# Re-combine the normalized and non-normalized data together for both train ans test sets (3 pts)
X_train_std.reset_index(drop=True, inplace=True)
X_train_nstd.reset_index(drop=True, inplace=True)
X_train = pd.concat([X_train_std,X_train_nstd], sort=False,axis=1)
X_test_std.reset_index(drop=True, inplace=True)
X_test_nstd.reset_index(drop=True, inplace=True)
X_test = pd.concat([X_test_std,X_test_nstd], sort=False,axis=1)

# Update the order within feature pool
feature_pool = list(X_train.columns)

The mean and standard deviation of training set: [1.14052757e+03 1.78283904e+02 2.57876132e-01 2.63935542e-02
 1.13368998e-01 8.60235841e-01 1.35467906e+01] [1.45658886e+03 2.30689376e+02 2.19284250e-01 3.75325557e-02
 6.34882426e-02 7.31163180e-02 9.13268176e+00]


In [133]:
X_test.to_csv('../data/X_test_scaled.csv', index=False)
X_train.to_csv('../data/X_train_scaled.csv', index=False)
y_train.to_csv('../data/Y_train_scaled.csv', index=False)
y_test.to_csv('../data/Y_test_scaled.csv', index=False)